Run `pip install .` first to install all dependencies.

In [1]:
!pip install gcsfs==2022.10.0
!pip install wandb==0.13.4
!pip install fastparquet
!pip install numpy
!pip install pandas
!pip install catboost

In [2]:
from catboost import CatBoostClassifier, Pool
import numpy as np

import gcsfs
import numpy as np
import pandas as pd

import wandb

from sklearn.preprocessing import MinMaxScaler


/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.13) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [3]:
name = "classical_size_features_accuracy_branch"

# connect to weights and biases
run = wandb.init(project="thesis", job_type="dataset-creation", entity="fbv")

dataset = wandb.Artifact(name=name, type="preprocessed_data")


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: karelze (fbv). Use `wandb login --relogin` to force relogin


In [4]:
# import google.auth
# from google.colab import auth
# # connect to google cloud storage
# auth.authenticate_user()
# credentials, _ = google.auth.default()
fs = gcsfs.GCSFileSystem(project="thesis")
#fs = gcsfs.GCSFileSystem(project="thesis", credentials=credentials)

/usr/lib/python3/dist-packages/google/auth/_default.py:66: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK. We recommend that most server applications use service accounts instead. If your application continues to use end user credentials from Cloud SDK, you might receive a "quota exceeded" or "API not enabled" error. For more information about service accounts, see https://cloud.google.com/docs/authentication/.
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


In [5]:
# reduce number of imported cols due to memory issues
columns = [
    "QUOTE_DATETIME",
    "ROOT",
    "EXPIRATION",
    "STRK_PRC",
    "OPTION_TYPE",
    "TRADE_SIZE",
    "TRADE_PRICE",
    "BEST_BID",
    "BEST_ASK",
    "ask_ex",
    "bid_ex",
    "bid_size_ex",
    "ask_size_ex",
    "price_all_lead",
    "price_all_lag",
    "price_ex_lead",
    "price_ex_lag",
    "buy_sell",
]


In [23]:
train = pd.read_parquet(
    f"gs://thesis-bucket-option-trade-classification/data/preprocessed/train_set_extended_60.parquet",
    engine="fastparquet",
    columns=columns,
)#.sample(frac=0.1)
val = pd.read_parquet(
    f"gs://thesis-bucket-option-trade-classification/data/preprocessed/val_set_extended_20.parquet",
    engine="fastparquet",
    columns=columns,
)#.sample(frac=0.1)

In [7]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler

In [8]:
# oe_option_type = OrdinalEncoder(
#     unknown_value=-1, dtype=int, handle_unknown="use_encoded_value"
# )
# oe_root = OrdinalEncoder(
#     unknown_value=-1, dtype=int, handle_unknown="use_encoded_value"
# )
# oe_issue_type = OrdinalEncoder(
#     unknown_value=-1, dtype=int, handle_unknown="use_encoded_value"
# )

scaler = MinMaxScaler(feature_range=[-1, 1])
# scaler = StandardScaler()
# scaler = QuantileTransformer(n_quantiles=256)

def transform(data: pd.DataFrame) -> pd.DataFrame:

    # # date features
    x = pd.DataFrame(data={"TRADE_PRICE": data["TRADE_PRICE"]}, index=data.index)


    

    # TODO: speak with caroline
    # x["bid_ex"].replace({0.0:np.NaN}, inplace=True)
    # x["ask_ex"].replace({0.0:np.NaN}, inplace=True)




    # size features
    x["bid_ask_size_ratio_ex"] = data["bid_size_ex"] / data["ask_size_ex"]
    x["rel_bid_size_ex"] = data["TRADE_SIZE"] / data["bid_size_ex"]
    x["rel_ask_size_ex"] = data["TRADE_SIZE"] / data["ask_size_ex"]
    x["depth_ex"] = data["bid_size_ex"] - data["ask_size_ex"]
    
    # classical
    mid_ex = 0.5 * (data["ask_ex"] + data["bid_ex"])
    mid_best = 0.5 * (data["BEST_ASK"] + data["BEST_BID"])
    spread_ex = (data["ask_ex"] - data["bid_ex"])
    spread_best = (data["BEST_ASK"] - data["BEST_BID"])
    
    x["prox_ex"] = (data["TRADE_PRICE"] - mid_ex) / (0.5 * spread_ex) 
    x["prox_best"] = (data["TRADE_PRICE"] - mid_best) / (0.5 * spread_best)  




    
    x["spread_ex"] = spread_ex
    x["spread_best"] = spread_best

    x["bid_ask_ratio_ex"] = data["bid_ex"] / data["ask_ex"]
    # x["price_norm_size"] = data["TRADE_PRICE"] / data["TRADE_SIZE"]
    x["price_rel_nbb"] = (data["BEST_ASK"] - data["TRADE_PRICE"]) / (data["BEST_ASK"] - mid_best)
    x["price_rel_nbo"] = (data["TRADE_PRICE"] - data["BEST_BID"]) / (mid_best - data["BEST_BID"])
    
    # x["price_eq_ask_eq_bid"] = (data["TRADE_PRICE"] == data["ask_ex"]) & (data["TRADE_PRICE"] == data["bid_ex"])
    
    # calculate change
    x["chg_ex_lead"] = data["TRADE_PRICE"] - data["price_ex_lead"]
    x["chg_ex_lag"] = data["TRADE_PRICE"] - data["price_ex_lag"]
    x["chg_all_lead"] = data["TRADE_PRICE"] - data["price_all_lead"]
    x["chg_all_lag"] = data["TRADE_PRICE"] - data["price_all_lag"]

    # calculate change own creation
    #x["chg_ex_lag_lead"] = data["price_ex_lag"] - data["price_ex_lead"]
    #x["chg_all_lag_lead"] = data["price_all_lag"] - data["price_all_lead"]

    # impute with -1
#    x.replace([np.inf, -np.inf], np.nan, inplace=True)

    # log transformed features
    x[
         [
             "ask_ex",
             "bid_ex",
             "BEST_ASK",
             "BEST_BID",
             "TRADE_PRICE",
             "price_all_lag",
             "price_all_lead",
             "price_ex_lag",
             "price_ex_lead",
             "TRADE_SIZE", 
             "bid_size_ex", 
             "ask_size_ex",
         ]
     ] = np.log1p(data[
         [
             "ask_ex",
             "bid_ex",
             "BEST_ASK",
             "BEST_BID",
             "TRADE_PRICE",
             "price_all_lag",
             "price_all_lead",
             "price_ex_lag",
             "price_ex_lead",
             "TRADE_SIZE", 
             "bid_size_ex", 
             "ask_size_ex"
         ]
     ]
     )

    # # scale to [-1, 1]
    # impute with zeros
    x.replace([np.inf, -np.inf], np.nan, inplace=True)
    x.fillna(0, inplace=True)

    # scale to [-1, 1]
    if not hasattr(scaler, "n_features_in_"):
        scaler.fit(x)
    x[x.columns] = scaler.transform(x)


    x["buy_sell"] = data["buy_sell"]
    return x


In [24]:
min_max_scaler = MinMaxScaler(feature_range=[-1, 1])


def transform(data: pd.DataFrame) -> pd.DataFrame:

    # # date features
    x = pd.DataFrame(data={"TRADE_PRICE": data["TRADE_PRICE"]}, index=data.index)

    # x["date_month_sin"] = np.sin(2 * np.pi * data["QUOTE_DATETIME"].dt.year / 12)
    # x["date_month_cos"] = np.cos(2 * np.pi * data["QUOTE_DATETIME"].dt.year / 12)

    # seconds_in_day = 24 * 60 * 60
    # seconds = (
    #     data["QUOTE_DATETIME"] - data["QUOTE_DATETIME"].dt.normalize()
    # ).dt.total_seconds()

    # x["date_time_sin"] = np.sin(2 * np.pi * seconds / seconds_in_day)
    # x["date_time_cos"] = np.cos(2 * np.pi * seconds / seconds_in_day)

    # option features
    # x["ttm"] = (
    #     data["EXPIRATION"].dt.to_period("M") - data["QUOTE_DATETIME"].dt.to_period("M")
    # ).apply(lambda x: x.n)
    # x[["myn", "day_vol"]] = data[["myn", "day_vol"]]
    # x["log_strk_prc"] = np.log1p(data["STRK_PRC"])

    # binarize
    # "bin_OPTION_TYPE", "bin_issue_type", "bin_ROOT",

    # size features
    x["bid_ask_size_ratio_ex"] = data["bid_size_ex"] / data["ask_size_ex"]
    x["rel_bid_size_ex"] = data["TRADE_SIZE"] / data["bid_size_ex"]
    x["rel_ask_size_ex"] = data["TRADE_SIZE"] / data["ask_size_ex"]
    # x[["TRADE_SIZE", "bid_size_ex", "ask_size_ex"]] = data[
    #     ["TRADE_SIZE", "bid_size_ex", "ask_size_ex"]
    # ]

    # classical
    mid_ex = 0.5 * (data["ask_ex"] + data["bid_ex"])
    mid_best = 0.5 * (data["BEST_ASK"] + data["BEST_BID"])
    x["rel_ask_ex"] = (data["TRADE_PRICE"] - mid_ex) / (data["ask_ex"] - mid_ex)
    x["rel_bid_ex"] = (mid_ex - data["TRADE_PRICE"]) / (mid_ex - data["bid_ex"])
    x["BEST_rel_bid"] = (data["TRADE_PRICE"] - mid_best) / (data["BEST_ASK"] - mid_best)
    x["BEST_rel_ask"] = (mid_best - data["TRADE_PRICE"]) / (mid_best - data["BEST_BID"])
    x["bid_ask_ratio_ex"] = data["bid_ex"] / data["ask_ex"]

    x["chg_ex_lead"] = data["TRADE_PRICE"] - data["price_ex_lead"]
    x["chg_ex_lag"] = data["TRADE_PRICE"] - data["price_ex_lag"]
    x["chg_all_lead"] = data["TRADE_PRICE"] - data["price_all_lead"]
    x["chg_all_lag"] = data["TRADE_PRICE"] - data["price_all_lag"]

    # log transformed features
    x[
         [
             "ask_ex",
             "bid_ex",
             "BEST_ASK",
             "BEST_BID",
             "TRADE_PRICE",
             "price_all_lag",
             "price_all_lead",
             "price_ex_lag",
             "price_ex_lead",
             "TRADE_SIZE", 
             "bid_size_ex", 
             "ask_size_ex",
         ]
     ] = np.log1p(data[
         [
             "ask_ex",
             "bid_ex",
             "BEST_ASK",
             "BEST_BID",
             "TRADE_PRICE",
             "price_all_lag",
             "price_all_lead",
             "price_ex_lag",
             "price_ex_lead",
             "TRADE_SIZE", 
             "bid_size_ex", 
             "ask_size_ex"
         ]
     ]
    )

    # impute with zeros
    x.replace([np.inf, -np.inf], np.nan, inplace=True)
    x.fillna(0, inplace=True)

    # scale to [-1, 1]
    if not hasattr(min_max_scaler, "n_features_in_"):
        min_max_scaler.fit(x)
    x[x.columns] = min_max_scaler.transform(x)

    # https://stackoverflow.com/questions/70727291/how-do-i-know-whether-a-sklearn-scaler-is-already-fitted-or-not

    # if not hasattr(oe_option_type, "n_features_in_"):
    #     oe_option_type.fit(data["OPTION_TYPE"].astype(str).values.reshape(-1, 1))
    # x["bin_option_type"] = oe_option_type.transform(
    #     data["OPTION_TYPE"].astype(str).values.reshape(-1, 1)
    # )

    # if not hasattr(oe_root, "n_features_in_"):
    #     oe_root.fit(data["ROOT"].astype(str).values.reshape(-1, 1))
    # x["bin_root"] = oe_root.transform(data["ROOT"].astype(str).values.reshape(-1, 1))

    # if not hasattr(oe_issue_type, "n_features_in_"):
    #     oe_issue_type.fit(data["issue_type"].astype(str).values.reshape(-1, 1))
    # x["bin_issue_type"] = oe_issue_type.transform(
    #     data["issue_type"].astype(str).values.reshape(-1, 1)
    # )

    # x.replace([np.inf, -np.inf], np.nan, inplace=True)

    x["buy_sell"] = data["buy_sell"]
    return x

In [25]:
X_train = transform(train)
y_train = X_train.buy_sell
X_train.drop(columns=["buy_sell"], inplace=True)

del train

In [26]:
X_val = transform(val)
y_val = X_val.buy_sell
X_val.drop(columns=["buy_sell"], inplace=True)
del val

In [28]:
params = {
        "od_type": "Iter",
        "logging_level": "Silent",
        "loss_function": "Logloss",
        "task_type": "GPU",
        "cat_features": None,
        "random_seed": 42,
        "eval_metric":"Accuracy",
       "iterations":10000,
       "early_stopping_rounds":100,
       "grow_policy":"Lossguide",
       "border_count": 254,
       'learning_rate': 0.013292918943162165, 
       'depth': 10, 
}


weight = np.geomspace(0.001, 1, num=len(y_train))
# keep ordering of data
timestamp = np.linspace(0, 1, len(y_train))

model = CatBoostClassifier(**params)
train_pool = Pool(X_train, y_train,weight = weight, timestamp=timestamp)
model.fit(train_pool, eval_set=(X_val,y_val))

print(model.score(X_val, y_val))

0.5732721132572086


In [12]:
X_train.isna().sum()

TRADE_PRICE              0
bid_ask_size_ratio_ex    0
rel_bid_size_ex          0
rel_ask_size_ex          0
depth_ex                 0
prox_ex                  0
prox_best                0
spread_ex                0
spread_best              0
bid_ask_ratio_ex         0
price_rel_nbb            0
price_rel_nbo            0
chg_ex_lead              0
chg_ex_lag               0
chg_all_lead             0
chg_all_lag              0
ask_ex                   0
bid_ex                   0
BEST_ASK                 0
BEST_BID                 0
price_all_lag            0
price_all_lead           0
price_ex_lag             0
price_ex_lead            0
TRADE_SIZE               0
bid_size_ex              0
ask_size_ex              0
dtype: int64

In [22]:
feature_importance = model.get_feature_importance(prettified=True)
feature_importance

,Feature Id,Importances
0,ask_size_ex,15.630816
1,prox_ex,14.605124
2,prox_best,12.880234
3,bid_size_ex,11.974249
4,price_rel_nbo,10.117559
5,bid_ask_size_ratio_ex,8.559225
6,price_rel_nbb,8.010360
7,rel_bid_size_ex,4.435536
8,rel_ask_size_ex,2.829633
9,spread_ex,2.181813


In [14]:
X_train.head()

,TRADE_PRICE,bid_ask_size_ratio_ex,rel_bid_size_ex,rel_ask_size_ex,depth_ex,prox_ex,prox_best,spread_ex,spread_best,bid_ask_ratio_ex,...,bid_ex,BEST_ASK,BEST_BID,price_all_lag,price_all_lead,price_ex_lag,price_ex_lead,TRADE_SIZE,bid_size_ex,ask_size_ex
0,-0.693407,-0.999869,-0.999974,-0.999886,0.001115,-0.449015,-0.221304,-0.337931,-0.997495,-0.638095,...,-0.704866,-0.814617,-0.704869,-0.768801,-0.815041,-0.686569,-0.714617,-0.666454,-0.151770,-0.471112
1,-0.561888,-1.000000,-1.000000,-1.000000,0.000005,-0.449594,-0.220000,-0.339080,-0.997495,-1.000000,...,-1.000000,-0.742978,-0.576987,-0.650515,-0.720412,-0.554139,-0.538233,-0.666454,-1.000000,-1.000000
2,-0.308833,-0.999980,-0.999943,-0.999962,0.000005,-0.449981,-0.222029,-0.337356,-0.997494,-0.610526,...,-0.308892,-0.587469,-0.308899,-0.460409,-0.557116,-0.311609,-0.314565,-0.366329,-0.008573,-0.008573
3,-0.952182,-1.000000,-1.000000,-0.999974,-0.000855,-0.448899,-0.221217,-0.337644,-0.997494,-1.000000,...,-1.000000,-0.963437,-1.000000,-0.969651,-0.975721,-0.961282,-0.962538,-0.666454,-1.000000,-0.224192
4,-0.940858,-0.999832,-0.999998,-0.999991,0.029575,-0.450753,-0.223478,-0.337931,-0.997496,-0.777778,...,-0.938145,-0.944868,-0.927274,-0.934833,-0.947866,-0.916862,-0.919561,-0.593142,0.410381,0.040824


In [15]:
# 0.7333797810178145 (logs applied) to     x["bid_ask_size_ratio_ex"] x["rel_bid_size_ex"]   x["rel_ask_size_ex"] 
# 0.7331946683482763 (all from above + log returns)
# 0.734859665271541 (all above + compare mids, ask, and bid at exchange with nation wide)
# 0.7349013664773161 (all above + trade_price == bid_ex == ask_ex)
# 0.7351464881990674 (price normed size)
# 0.7352807457396116 (all above +  x["price_rel_nbo"] +  x["price_rel_nbb"])
# 0.74296292 (some removed, see above, max. iterations reached. Actually there is no reason to stop ensemble)
# 0.745053067 convert some ratios to percentages (all other things the same as above)
# 0.7434765583282902 without any scaler, zero imputer, symmetric tree
# 0.7445394305242655 "grow_policy" = "Lossguide" -> trained for 5000 iterations
# 0.7450428963013065 impute with -999 instead of 0 -> trained for 5000 iterations
# 0.7436240382023729 TODO: ask Caroline: What happens with LR if bid_ex = 0 or ask_ex is?   x["bid_ex"].replace({0.0:np.NaN}, inplace=True) and  x["ask_ex"].replace({0.0:np.NaN}, inplace=True)
# 0.7447164063731647 chg from previous trade to successive trade
# 0.7445821488326205 chg from previous trade + no trade indicator
# 0.7476558328290199 "spread" feature ex
# 0.747136093410701 "spread" feature best ex + fixed typo
# 0.7475317463142745 removed features with low importance + impute with -1

## Sanity Check against `lightgbm`

In [16]:
%%script false --no-raise-error
!pip install lightgbm

In [17]:
%%script false --no-raise-error
import lightgbm as lgb

In [18]:
%%script false --no-raise-error
lgb_params = {
                    'objective':'binary',
                    'boosting_type':'gbdt',
                    'metric':None,
                    'n_jobs':-1,
                    'learning_rate':0.01,
                    'num_leaves': 2**8,
                    'max_depth':-1,
                    'tree_learner':'serial',
                    'colsample_bytree': 0.7,
                    'subsample_freq':1,
                    'subsample':0.7,
                    'n_estimators':200,
                    'max_bin':255,
                    'verbose':-1,
                    'seed': 42,
                    'eval_metric':'accuracy',
                    "device": "cpu",
                }

In [19]:
%%script false --no-raise-error
# create dataset for lightgbm
# if you want to re-use data, remember to set free_raw_data=False
lgb_train = lgb.Dataset(X_train, y_train,weight=X_train.index.values, free_raw_data=False)
lgb_eval = lgb.Dataset(X_val, y_val, reference=lgb_train, free_raw_data=False)

In [20]:
%%script false --no-raise-error
bst = lgb.LGBMClassifier(**lgb_params)
bst = lgb.train(lgb_params,
                train_set=lgb_train,
                valid_sets=lgb_eval,
                early_stopping_rounds=100
)

In [21]:
%%script false --no-raise-error
pred = bst.predict(X_val)
pred = np.rint(pred)
pred[pred==0]=-1
print((y_val == pred).mean())